In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import scipy as sc
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn import preprocessing
from keras.models import Sequential
from tensorflow.keras import layers

In [3]:
data=pd.read_csv("/content/drive/MyDrive/Timeseries_datasets/rainfall.csv")
data.head()

,date,rainfall
0,01-01-2010,2.9
1,02-01-2010,10.2
2,03-01-2010,0.9
3,04-01-2010,48.6
4,05-01-2010,9.6


In [4]:
data.shape

(3774, 2)

In [5]:
train_split=0.9
train_index=int(data.shape[0]*train_split)
data_past=30
data_future=5

In [6]:
training_data=data.iloc[:train_index].values
validation_data=data.iloc[train_index:].values

In [7]:
print("Training data dimensions:",np.shape(training_data))
print("Validation data dimensions:",np.shape(validation_data))

Training data dimensions: (3396, 2)
Validation data dimensions: (378, 2)


In [8]:
x_train=[]
y_train=[]
for i in range(len(training_data) - data_past - data_future + 1):
  x_train.append(training_data[i:i+data_past,1])
  y_train.append(training_data[i+data_past:i+data_past+data_future,1])

In [9]:
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.asarray(x_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')

In [10]:
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

In [11]:
print("This data can be framed as 1 sample with ",x_train.shape[0]," time steps and ",x_train.shape[1]," features")

This data can be framed as 1 sample with  3362  time steps and  30  features


In [12]:
x_train=x_train.reshape(1,x_train.shape[0],x_train.shape[1])
y_train=y_train.reshape(1,y_train.shape[0],y_train.shape[1])

In [13]:
EPOCHS = 20
BATCH_SIZE = 32
model = Sequential()
model.add(LSTM(units= 30, return_sequences=True,input_shape=(3362,30)))
model.add(Dense(units = data_future, activation='relu'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['acc'])

model.build(x_train.shape)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3362, 30)          7320      
_________________________________________________________________
dense (Dense)                (None, 3362, 5)           155       
Total params: 7,475
Trainable params: 7,475
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train, epochs=EPOCHS,batch_size=BATCH_SIZE)

In [ ]:
# using Bi-LSTM
EPOCHS = 20
BATCH_SIZE = 32

model = Sequential()
model.add(Bidirectional(LSTM(units=30, return_sequences=True, input_shape = (3362  , 30))))
model.add(Dropout(0.2))
model.add(LSTM(units= 30, return_sequences=True,input_shape=(30,1)))
model.add(Dropout(0.2))
model.add(LSTM(units= 30, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units= 30))
model.add(Dropout(0.2))
model.add(Dense(units = data_future, activation='relu'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['acc'])

In [ ]:
model.build(x_train.shape)
model.summary()
model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

In [ ]:
print(x_train.shape)
print(x_train[0].shape)
pred=np.reshape(x_train[0],(1,x_train.shape[1],x_train.shape[2]))
print(pred.shape)

In [ ]:
model.predict(pred)